In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from my_functions import *

In [ ]:
tcurves = load_tcurves(load_filter_tags())
w_central = central_wavelength(tcurves)

In [ ]:
split_converter = lambda s: np.array(s.split()).astype(float)
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))
tile_name = lambda s: s[:8]

cat = pd.read_csv('csv/minijpas.Flambda_photoz_gaia_3.csv', sep=',', header=1,
    converters={0: tile_name, 1: split_converter, 2: split_converter, 3: sum_flags,
    4: sum_flags})

cat = cat[np.array([len(x) for x in cat['FLUX_AUTO']]) != 0] # Drop flx == 0
cat = cat[(cat.FLAGS == 0) & (cat.MASK_FLAGS == 0)] # Drop flagged
cat = cat.reset_index()

pm_flx = np.stack(cat['FLUX_AUTO'].to_numpy()).T * 1e-19
pm_err = np.stack(cat['FLUX_RELERR_AUTO'].to_numpy()).T * pm_flx

In [ ]:
# Function to drop sources
def make_masks(cat, nb_ind):
    # Mask sources with high photoz odds
    #mask_pz_odds = cat['odds'] < 0.98
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_error'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_error'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_error'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    # Mask sources with SNR < 5 in the selected NB
    mask_snr = (pm_err / pm_flx)[nb_ind, :] < 0.2
    # Mask too bright gSDSS
    mask_b = pm_flx[-3, :] > mag_to_flux(15, 4750)
    
    mask_total = mask_pmotion & mask_snr & mask_b # & mask_pz_odds
    # print('Total masked: {}'.format(len(np.where(~mask_total)[0])))
    return mask_total.to_numpy()

In [ ]:
N_nb = 6 # Number of nb on each side of the central one
ew0min = 30

N_sources = len(cat['FLUX_AUTO'])
print(N_sources)

In [ ]:
nb_c_min = 6
nb_c_max = 50
line = np.zeros((nb_c_max - nb_c_min, N_sources)).astype(bool)
cont_est_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
cont_err_Arr = np.zeros((nb_c_max - nb_c_min, N_sources))
i = 0
for nb_c in range(nb_c_min, nb_c_max):
    z_nb = w_central[nb_c] / 1215.67 - 1
    fwhm = nb_fwhm(nb_c, tcurves=tcurves)
    cont_est, cont_err = stack_estimation(pm_flx, pm_err, nb_c, N_nb, False)
    line[i] = (
        (pm_flx[nb_c] - cont_est > 3 * (cont_err**2 + pm_err[nb_c]**2) ** 0.5)
        & (pm_flx[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )
    cont_est_Arr[i] = cont_est
    cont_err_Arr[i] = cont_err
    i += 1

In [ ]:
## QSO_find_lines TEST
qso_LAE_Arr, qso_single_Arr, line_lya, line_other =\
    QSO_find_lines(pm_flx, pm_err, 6, 50, 30, 15, 6)

In [ ]:
print('QSO candidates found : {}'.format(len(qso_LAE_Arr)))
print('QSO candidates single: {}'.format(len(qso_single_Arr)))

In [ ]:
cat.head()

In [ ]:
j = 0
for src in qso_LAE_Arr:
    fig = plt.figure(figsize=(10, 8))
    ax = plot_JPAS_source(pm_flx[:, src], pm_err[:, src])

    for l in line_other[src]:
        ax.errorbar(w_central[l],
            cont_est_Arr[l - 6, src],
            yerr=cont_err_Arr[l - 6, src],
            fmt='^', c='C8', markersize=10, capsize=3, capthick=3, elinewidth=4)

    ax.errorbar(w_central[line_lya[src]],
        cont_est_Arr[line_lya[src] - 6, src],
        yerr=cont_err_Arr[line_lya[src] - 6, src],
        fmt='^', c='C4', markersize=10, capsize=3, capthick=3, elinewidth=4)

    z_src = w_central[line_lya[src]] / 1215.67 - 1
    plt.show()
    print('z_nb = {0:0.2f}'.format(z_src))
    print('z_sp = {0:0.2f} ({1})'.format(cat.zsp[src], cat.spCl[src]))
    print('z_pz = {0:0.2f}, odds = {1:0.2f}'.format(cat.PHOTOZ[src], cat.ODDS[src]))

    j += 1
    if j == 20: break